In [1]:
from tqdm import tqdm
from sklearn.decomposition import NMF
import pandas as pd
from scipy import sparse
import numpy as np

# read data
movies = pd.read_csv("data/ml-20m/movies.csv")
ratings = pd.read_csv("data/ml-20m/ratings.csv")

# join
ratings_joined = pd.merge(ratings, movies)

# ratingsをsparse matrixに変換して横持ちにする
action_adventure_ratings = ratings_joined.query("genres.str.contains('Action') or genres.str.contains('Adventure')", 
                                                engine='python').reset_index(drop=True)
# indexing ids
# userid
userid_unique = pd.Series(action_adventure_ratings["userId"].unique())
index_userid_dict = userid_unique.to_dict()
# inverse
userid_index_dict = dict(map(reversed, index_userid_dict.items()))

# itemid
itemid_unique = pd.Series(action_adventure_ratings["movieId"].unique())
index_itemid_dict = itemid_unique.to_dict()
# inverse
itemid_index_dict = dict(map(reversed, index_itemid_dict.items()))

action_adventure_ratings["user_id"] = action_adventure_ratings["userId"].map(userid_index_dict)
action_adventure_ratings["item_id"] = action_adventure_ratings["movieId"].map(itemid_index_dict)

# reindexしたidを使って、アイテムとジャンルの対応が取れるdictを作る
itemid_genres_dict = action_adventure_ratings[['item_id', 'genres']].set_index('item_id')['genres'].to_dict()

In [2]:
import cloudpickle
X_train = cloudpickle.load(open("output/ML-20M-X_train.pkl","rb"))
X_test = cloudpickle.load(open("output/ML-20M-X_test.pkl","rb"))

In [3]:
# aggregateのtrainをactionとadventureに分離する
# actionの列
action_columns = [v for v in range(X_train.shape[1]) if 'Action' in itemid_genres_dict[v]]
# adventureの列
adventure_columns = [v for v in range(X_train.shape[1]) if 'Adventure' in itemid_genres_dict[v]]

# 選んだカラムに応じてとってくる
action_train = X_train[:, action_columns]
adventure_train = X_train[:, adventure_columns]

# adventureのみ、アイテムidのconcatとの対応関係が必要なので辞書として持っておく
adventure_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'Adventure' in itemid_genres_dict[v]:
        adventure_concat_itemid_dict[v] = count
        count += 1

In [4]:
# collective matrix factorizationのパッケージ
!pip3 install cmfrec

In [5]:
# アイテムidのconcatとの対応関係が必要なので辞書として持っておく
action_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'Action' in itemid_genres_dict[v]:
        action_concat_itemid_dict[v] = count
        count += 1
# inverse
inverse_action_concat_itemid_dict = dict(map(reversed, action_concat_itemid_dict.items()))

adventure_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'Adventure' in itemid_genres_dict[v]:
        adventure_concat_itemid_dict[v] = count
        count += 1
# inverse
inverse_adventure_concat_itemid_dict = dict(map(reversed, adventure_concat_itemid_dict.items()))

In [6]:
# それぞれにアクションしていないユーザを削る
# 全ユーザと、削ったあとでの対応関係を辞書として持っておく
action_train_selected = action_train[action_train.getnnz(1)>0]
adventure_train_selected = adventure_train[adventure_train.getnnz(1)>0]

action_train_action_users = {}
action_users = action_train.getnnz(1)>0
count = 0
for i in range(action_train.shape[0]):
    if action_users[i]:
        action_train_action_users[i] = count
        count += 1

# inverse
inverse_action_train_action_users = dict(map(reversed, action_train_action_users.items()))

adventure_train_action_users = {}
adventure_users = adventure_train.getnnz(1)>0
count = 0
for i in range(adventure_train.shape[0]):
    if adventure_users[i]:
        adventure_train_action_users[i] = count
        count += 1

# inverse
inverse_adventure_train_action_users = dict(map(reversed, adventure_train_action_users.items()))

In [7]:
# actionだけでALSをまず行う
# mediateでの結果をそのまま持ってくる
import cloudpickle
action_ALS = cloudpickle.load(open('output/ML-20M-action_ALS.pkl', 'rb'))

In [8]:
# side informationとしてaction_NMFで得られたベクトルを使う
user_attributes = pd.DataFrame(action_ALS.user_factors)
user_attributes['UserId'] = user_attributes.index
# useridをconcatの次元に戻す
user_attributes['UserId'] = user_attributes['UserId'].map(inverse_action_train_action_users)

item_attributes = pd.DataFrame(action_ALS.item_factors)
item_attributes['ItemId'] = item_attributes.index
# itemidをconcatの次元に戻す
item_attributes['ItemId'] = item_attributes['ItemId'].map(inverse_action_concat_itemid_dict)

In [9]:
# X_trainから縦持ちに復元する(X_trainが既に横持ちなので)
userid_stacked = []
itemid_stacked = []
ratings_stacked = []

for i in tqdm(range(X_train.shape[0])):
    ratings = X_train[i, :].data
    item_idx = X_train[i, :].indices
    for idx, v in enumerate(item_idx):
        if 'Adventure' in itemid_genres_dict[v]:
            userid_stacked.append(i)
            itemid_stacked.append(v)
            ratings_stacked.append(ratings[idx])

adventure_ratings_stacked = pd.DataFrame()
adventure_ratings_stacked['UserId'] = userid_stacked
adventure_ratings_stacked['ItemId'] = itemid_stacked
adventure_ratings_stacked['Rating'] = ratings_stacked

100%|██████████| 138389/138389 [00:41<00:00, 3306.64it/s]


In [10]:
adventure_ratings_stacked.head(5)

,UserId,ItemId,Rating
0,0,94,4.0
1,1,26,5.0
2,2,185,5.0
3,3,189,4.0
4,4,0,3.0


In [11]:
from cmfrec import CMF

# fitting a model and making some recommendations
recommender = CMF(k=20, k_main=3, k_user=2, k_item=1, reg_param=1e-4)
recommender.fit(ratings=adventure_ratings_stacked, user_info=user_attributes, item_info=item_attributes,
                cols_bin_user=None, cols_bin_item=None)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 1.042674
  Number of iterations: 416
  Number of functions evaluations: 434


In [12]:
# 評価対象のユーザ
test_adventure_pos_items_dict = {}
for i in tqdm(range(X_test.shape[0])):
    # trainでadventureにアクションしていないユーザに
    rated_items = X_train[i, :].indices
    if len([v for v in rated_items if 'Adventure' in itemid_genres_dict[v]]) == 0:
        # X_testの中でstoreしているアイテムが0以上のユーザに
        if X_test[i, :].nnz > 0:
            test_items = []
            selected_user_ratings = X_test[i, :]
            value_indices = selected_user_ratings.indices
            sorted_indices = np.argsort(-X_test[i, :].toarray())[0]
            # valueがあるアイテムのジャンルがadventureの場合に
            for v in sorted_indices[:len(value_indices)]:
                if 'Adventure' in itemid_genres_dict[v]:
                    test_items.append(v)
            if len(test_items) > 0:
                test_adventure_pos_items_dict[i] = test_items

100%|██████████| 138389/138389 [00:26<00:00, 5173.11it/s]


In [13]:
recommender.A.shape

(138389, 25)

In [14]:
from lib.recommend_util import ndcg
ndcgs = {
    'ndcg5':  [],
    'ndcg10':  [],
    'ndcg20':  [],
    'ndcg50':  [],
    'ndcg100':  []
}
count = 0
for userid, pos_items in tqdm(test_adventure_pos_items_dict.items()):
    pos_items = np.array(pos_items)
    try:
        recommended_items = np.array(recommender.topN(user=userid, n=100))
        ndcgs['ndcg5'].append(ndcg(recommended_items[:5], pos_items))
        ndcgs['ndcg10'].append(ndcg(recommended_items[:10], pos_items))
        ndcgs['ndcg20'].append(ndcg(recommended_items[:20], pos_items))
        ndcgs['ndcg50'].append(ndcg(recommended_items[:50], pos_items))
        ndcgs['ndcg100'].append(ndcg(recommended_items[:100], pos_items))
    except:
        count += 1
        # 推薦できないユーザの場合は無条件で0を入れる
        ndcgs['ndcg5'].append(0)
        ndcgs['ndcg10'].append(0)
        ndcgs['ndcg20'].append(0)
        ndcgs['ndcg50'].append(0)
        ndcgs['ndcg100'].append(0)

100%|██████████| 18319/18319 [00:18<00:00, 985.54it/s] 


In [15]:
print("ndcg@5: {}".format(np.mean(ndcgs['ndcg5'])))
print("ndcg@10: {}".format(np.mean(ndcgs['ndcg10'])))
print("ndcg@20: {}".format(np.mean(ndcgs['ndcg20'])))
print("ndcg@50: {}".format(np.mean(ndcgs['ndcg50'])))
print("ndcg@100: {}".format(np.mean(ndcgs['ndcg100'])))

ndcg@5: 0.028701190918832862
ndcg@10: 0.049119128646164634
ndcg@20: 0.07647956571300699
ndcg@50: 0.11911942405323482
ndcg@100: 0.17627805246960312
